# TMA Capacitor Bank discharge vs. Acceleration profiles 
# [SITCOM-1223]

Laura Toribio San Cipriano
"Feb. 14, 2024"

https://jira.lsstcorp.org/browse/SITCOM-1223

**Description:**
Continue the analysis started in SITCOM-1146. Based on the input from a team review, focusing on aspects below. Another ticket opened to investigate current draw profiles.

- Relationship of min power supply voltage and distance of slew
- Explore when Az and El moved at same time, rather than separately
- Look at Azimuth acceleration vs. power supply voltage

Results and analysis to be added to SITCOMTN-110

### Setup

In [36]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

from lsst_efd_client import EfdClient, resample
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState

from astropy.time import Time, TimeDelta
from astropy.table import Table
from scipy.interpolate import UnivariateSpline

In [38]:
client = EfdClient('usdf_efd')

In [39]:
tma_power_topic = 'lsst.sal.MTMount.mainPowerSupply'
tma_el_topic = 'lsst.sal.MTMount.elevation'
tma_az_topic = 'lsst.sal.MTMount.azimuth'

### Get Data

Getting data corresponding to these:
 * [BLOCK-160](https://jira.lsstcorp.org/browse/BLOCK-160)
 * [BLOCK-164](https://jira.lsstcorp.org/browse/BLOCK-164)

In [40]:
obs_days = [20231211, 20231212, 20231213, 20231214, 20231215, 20231218]

In [41]:
def get_events(dayObs):
    """
    Identify slew events for a given dayObs
    """
    eventMaker = TMAEventMaker()
    events = eventMaker.getEvents(dayObs)
    
    # Get lists of slew and track events
    slews = [e for e in events if e.type == TMAState.SLEWING]
    tracks = [e for e in events if e.type == TMAState.TRACKING]
    print(f"{dayObs}: Found {len(slews)} slews and {len(tracks)} tracks")
    return slews

In [42]:
def get_acc_jerk(time, velocity):
    """
    From velocity data, calculate acceleration and jerk
    """
    smoothing_factor=0.2
    kernel_size = 5
    kernel = np.ones(kernel_size) / kernel_size
    
    VelSpline = UnivariateSpline(time, velocity, s=0)
    smoothedVel = np.convolve(VelSpline(time), kernel, mode='same')
    VelSpline = UnivariateSpline(time, smoothedVel, s=smoothing_factor)

    AccSpline = VelSpline.derivative(n=1)
    smoothedAcc = np.convolve(AccSpline(time), kernel, mode='same')
    AccSpline = UnivariateSpline(time, smoothedAcc, s=smoothing_factor)
    
    JerkSpline = AccSpline.derivative(n=1) 

    return VelSpline(time), AccSpline(time), JerkSpline(time)

In [43]:
async def data_per_slew(i, slew, debug=False):
    """
    get TMA velocity data and power data for a given slew.

    Calculate maximum velocity, acceleration and jerk values and minimum power draw
    """
    data = [i, slew.dayObs, slew.duration, slew.begin, slew.end]
    power_df = await client.select_time_series(tma_power_topic, '*', slew.begin, slew.end)
    data.append(np.min(power_df.powerSupplyVoltage))
    
    az = await client.select_time_series(tma_az_topic, ['*'], slew.begin, slew.end)
    el = await client.select_time_series(tma_el_topic, ['*'], slew.begin, slew.end)

    AzVel, AzAcc, AzJerk = get_acc_jerk(az.timestamp, az.actualVelocity)
    ElVel, ElAcc, ElJerk = get_acc_jerk(el.timestamp, el.actualVelocity)

    for var in [AzVel, AzAcc, AzJerk, ElVel, ElAcc, ElJerk]:
        data.append(np.max(np.abs(var)))
    if debug:
        return az, el, power_df, data
    else:
        return data

In [44]:
Data = []
failed = []
data_names = ['slew_id','dayobs','duration','begin','end','min_power','az_max_vel','az_max_acc','az_max_jerk','el_max_vel','el_max_acc','el_max_jerk']
for dayObs in obs_days:
    try:
        slews = get_events(dayObs)
    except:
        print(dayObs)
    for i, slew in enumerate(slews):
        try:
            d = await data_per_slew(i, slew)
            Data.append(d)
        except Exception as e:
            failed.append(i)
            #print(i, e)
df = pd.DataFrame(np.vstack(Data),columns=data_names)
len(df)

20231211: Found 528 slews and 449 tracks


20231212: Found 211 slews and 124 tracks


20231213: Found 75 slews and 2 tracks


20231214: Found 398 slews and 265 tracks


20231215: Found 239 slews and 126 tracks


20231218: Found 227 slews and 39 tracks


1580

In [20]:
#Remove extreme outliers
df = df[(df.el_max_jerk < 100)&(df.duration<3000)]

NameError: name 'df' is not defined